In [1]:
import ssid
import quakeio
from ssid import modes
from utilities import Config, extract_channels, list_files, print_modes, mode_statistics

### Data inputs

In [2]:
directory = "uploads/vincent"
pattern  = "*.[zZ][iI][pP]"

In [3]:
data_conf = Config()
data_conf.d = 8 # decimation factor

# # PAINTER RIO DELL TRANSVERSE (CASE 1)
# data_conf.inputs  = [17,3,20]
# data_conf.outputs = [9,7,4]
# # PAINTER RIO DELL LONGITUDINAL (CASE 2)
# data_conf.inputs  = [15,1,18]
# data_conf.outputs = [11]

# # HWY8/MELOLAND TRANSVERSE (CASE 1)
# data_conf.inputs  = [2],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND TRANSVERSE (CASE 2)
# data_conf.inputs  = [11,2,26],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND LONGITUDINAL (CASE 3)
# data_conf.inputs  = [12,4,25],
# data_conf.outputs = [27,8]

# # CROWLEY TRANSVERSE (CASE 1)
# data_conf.inputs = [4]
# data_conf.outputs = [6,7,9]
# # CROWLEY TRANSVERSE (CASE 2)
# data_conf.inputs = [6,4,9]
# data_conf.outputs = [7]
# # CROWLEY TRANSVERSE (CASE 3)
# data_conf.inputs = [4]
# data_conf.outputs = [7]
# # CROWLEY LONGITUDINAL (CASE 4)
# data_conf.inputs = [5]
# data_conf.outputs = [8]

# # RIDGECREST TRANSVERSE (CASE 1)
# data_conf.inputs  = [4]
# data_conf.outputs = [6,7,9]

# # SAMOA TRANSVERSE (CASE 1)
# data_conf.inputs  = [16]
# data_conf.outputs = [10,12]

# VINCENT TRANSVERSE (CASE 1)
data_conf.inputs  = [1,9,24]
data_conf.outputs = [2,5,6]

# CAPISTRANO TRANSVERSE (CASE 1)


# # HAYWARD TRANSVERSE (CASE 1)
# data_conf.inputs  = [2,7,25,18]
# data_conf.outputs = [13,15,23,20]
# # HAYWARD LONGITUDINAL
# data_conf.inputs  = [3,6,17],
# data_conf.outputs = [12,14,19]

# # BERNARDINO TRANSVERSE BENT 3 (CASE 1)
# data_conf.inputs  = [6]
# data_conf.outputs = [7,8]
# # BERNARDINO TRANSVERSE BENT 8 (CASE 2)
# data_conf.inputs  = [24]
# data_conf.outputs = [19,20]
# # BERNARDINO LONGITUDINAL BENT 3 (CASE 3)
# data_conf.inputs  = [4]
# data_conf.outputs = [10]
# # BERNARDINO LONGITUDINAL BENT 8 (CASE 4)
# data_conf.inputs  = [22]
# data_conf.outputs = [17,18]


### Method Inputs

### General Parameters
parameter | value
----------|-------
`p ` | number of output channels
`q ` | number of input channels
`nt` | number of timesteps
`dt` | timestep
`d`  | decimation (downsampling) factor

### Specific to Observer Kalman Identification (OKID)
parameter | value
----------|-------
`m`  | number of Markov parameters to compute (at most = nt)

### Specific to Eigensystem Realization Algorithm (ERA)
parameter | value
----------|-------
`no` | number of observability parameters
`nc` | number of controllability parameters
`r ` | reduced model order

### Specific to Data Correlations (DC)
parameter | value
----------|-------
`a`  | (alpha) number of additional block rows in Hankel matrix of correlation matrices
`b`  | (beta) number of additional block columns in Hankel matrix of correlation matrices
`l`  | initial lag
`g`  | lag (gap) between correlations

### Specific to System Realization with Information Matrix (SRIM)
parameter | value
----------|-------
`no` | number of steps used for identification (prediction horizon)

In [4]:
# Set Parameters
conf = Config()
conf.m  = 400
conf.no = 190
conf.nc = 190
conf.r  = 12
conf.a  = 0  
conf.b  = 0  
conf.l  = 10 
conf.g  = 3

In [5]:
files = list_files(directory, pattern)
event_modes = []
for file in files:
    print(file)

    try:
        event = quakeio.read(file, exclusions=["*filter*", "*date*"])
        print(file, event["peak_accel"])
        inputs, dt = extract_channels(event, data_conf["inputs"], decimate=data_conf["d"])
        outpts, dt = extract_channels(event, data_conf["outputs"], decimate=data_conf["d"])
        realization = ssid.system(method="srim", input=inputs, output=outpts, **conf)
    except Exception as e:
        print(e)
        continue
    ss_modes = modes.modes(realization,dt)
    event_modes.append(list(ss_modes.values()))
    print_modes(ss_modes)

event_frequencies = mode_statistics(event_modes, "freq")
event_frequencies

uploads/vincent/calexico_04apr2010_14607652_CE14406P.ZIP
uploads/vincent/calexico_04apr2010_14607652_CE14406P.ZIP -92.529
Spectral quantities:
         T         ζ
      1.599      0.008328 
      0.8827     0.01184  
      0.3757     0.01529  
      0.3016     0.001356 
      0.2959     0.001439 
      0.2868     0.003245 
Mean Period(s): 0.623696914079665
Standard Dev(s): 0.48398550343957547
uploads/vincent/chinohills_29jul2008_ce14406p.zip
'max_rows' must be at least 1.
uploads/vincent/ci38457511_ce14406p.zip
uploads/vincent/ci38457511_ce14406p.zip 117.752
No channels found
uploads/vincent/Inglewood_17May2009_CE14406P.ZIP


Only extracted 2 channels, 1 missing.


uploads/vincent/Inglewood_17May2009_CE14406P.ZIP -211.549
Spectral quantities:
         T         ζ
      0.9741     0.03007  
      0.6989     0.03999  
      0.5878     0.02382  
      0.4551     0.009126 
      0.3537     0.02349  
      0.2938     0.002252 
Mean Period(s): 0.5605565516872861
Standard Dev(s): 0.22947653278780175
uploads/vincent/lahabra_28mar2014_15481673_ce14406p.zip
uploads/vincent/lahabra_28mar2014_15481673_ce14406p.zip 67.536


Only extracted 2 channels, 1 missing.


Spectral quantities:
         T         ζ
      0.8756     0.006875 
      0.7778     0.001127 
      0.3054     0.007299 
      0.2516     0.004426 
      0.2105     0.03518  
      0.1807     0.05644  
Mean Period(s): 0.43360376081583335
Standard Dev(s): 0.28199443676078084
uploads/vincent/northridge_17jan1994_ce14406p.zip
uploads/vincent/northridge_17jan1994_ce14406p.zip 472.904
Spectral quantities:
         T         ζ
      3.029      0.7676   
      0.8937     0.3034   
      0.5025     0.1688   
      0.3366     0.08232  
Mean Period(s): 1.1905320408490752
Standard Dev(s): 1.0806869105621149
uploads/vincent/Whittier87_CE14406P.ZIP
uploads/vincent/Whittier87_CE14406P.ZIP -249.126
tuple index out of range


/mnt/c/Users/16507/Documents/GitHub/SystemIdentification/notebooks/utilities/config.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray([


[{'freq': 2.6618432641381147,
  'damp': 0.01529221745949096,
  'modeshape': array([-0.00128361+0.0150647j ,  0.00077459-0.00611908j,
          0.03723786-0.02550685j]),
  'cnd': 1.000414196059899,
  'distance': 0.16826669653195675},
 {'freq': 2.1973133062498653,
  'damp': 0.009125609110742776,
  'modeshape': array([ 0.06719501+0.04536164j, -0.02908757-0.00386029j,
          0.01383012-0.02843941j]),
  'cnd': 1.0013876633569962,
  'distance': -0.17557611208142365},
 {'freq': 3.2742257991176857,
  'damp': 0.007299045196383844,
  'modeshape': array([ 0.00220296+0.08680165j, -0.01625104+0.03650694j]),
  'cnd': 1.0041283296558279,
  'distance': 0.6215492614075939},
 {'freq': 1.9900336252857027,
  'damp': 0.16879553691048596,
  'modeshape': array([-0.02277218-0.06993248j,  0.00458776-0.01860259j,
          0.04034628+0.08722946j]),
  'cnd': 1.11170827975445,
  'distance': -0.3290035207144503}]

### Painter St Rio Dell Hwy 101 89324
![Painter St Rio Dell Hwy 101 89324 Sensors 1](./figures/89324_Page_1.png)
![Painter St Rio Dell Hwy 101 89324 Sensors 2](./figures/89324_Page_2.png)

### Meloland Hwy 8 El Centro 01336
![Meloland Hwy 8 El Centro 01336 Sensors](./figures/01336.png)

### Lake Crowley Hwy 395 54730
![Lake Crowley Hwy 395 54730 Sensors](./figures/54730.png)

### Hayward Hwy 580-238 58658
![Hayward Hwy 580-238 58658 Sensors 1](./figures/58658_Page_1.png)
![Hayward Hwy 580-238 58658 Sensors 2](./figures/58658_Page_2.png)

### San Bernardino I10-215
![San Bernardino I10-215 Sensors](./figures/23631.png)